In [13]:
import requests
import pandas as pd
import cv2
import urllib.request
import numpy as np
from matplotlib import pyplot as plt
#from keras.preprocessing import image

%matplotlib inline


baseurl = 'https://dog.ceo/api/'

listurl = baseurl + 'breeds/list/all'
randomappend = '/images/random/'
allimages = '/images'

data_raw_file ='../../Data/Raw/dogs.parquet'

#número de raças. 0 para todas as raças
maxBreeds = 3
#número máximo de fotos aleatórias de cada raça.
#0 para todas as fotos, máximo de 50 para amostra aleatória.
maxImages = 50

img_h, img_w = 64, 64 # Altura e largura das imagens

#breeds = []

# Busca Lista de Raças

In [14]:
obj = requests.get(listurl).json()
allbreeds = pd.DataFrame.from_dict(obj)
allbreeds.head()

,message,status
affenpinscher,[],success
african,[],success
airedale,[],success
akita,[],success
appenzeller,[],success


In [15]:
breeds = allbreeds.index
if maxBreeds > 0:
    breeds = allbreeds [0:maxBreeds].index
    

# Ler imagens

In [16]:
def ler_imagem(url):

# For remote, use urllib, as per "http://stackoverflow.com/questions/21061814"
    req = urllib.request.urlopen(url)

    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
# 0 = monocromatic
    img = cv2.imdecode(arr,0)
    imres = cv2.resize(img, (img_h, img_w), interpolation=cv2.INTER_CUBIC)


# Plot
#    plt.figure()  
#    plt.imshow(imres)
    return imres

In [25]:
imageurls = []
dogimages = []
rotulos = []
#dataset = np.ndarray((len(breeds) * maxImages, img_h, img_w, 3), dtype=np.uint8)
i = 0

for breed in breeds:
    filesurl = ''
    if maxImages > 0:
        filesurl = baseurl + 'breed/' + breed + randomappend + str(maxImages)
    else:
        filesurl = baseurl + 'breed/' + breed + allimages
    imageurl = requests.get(filesurl).json()
    print(imageurl)
    breedimages = pd.DataFrame.from_dict(imageurl['message'])
    imageurls.append(breedimages)
    for images in breedimages.values:
        img = ler_imagem(images[0])
#        dataset[i] = img
        rotulos.append(breed)
        dogimages.append(img.flatten())
        i = i+1

{'message': ['https://images.dog.ceo/breeds/affenpinscher/n02110627_10185.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10225.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10787.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_10859.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11262.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11263.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11365.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11584.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11620.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11663.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_11811.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_12003.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_12070.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n02110627_12077.jpg', 'https://images.dog.ceo/breeds/affenpinscher/n021

In [19]:
#dataset.shape

In [22]:
df = pd.DataFrame(dogimages)
df['breed'] = rotulos

In [23]:
df

,0,1,2,3,4,5,6,7,8,9,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,breed
0,178,127,73,184,85,108,99,109,39,153,...,69,85,104,64,95,69,58,65,91,affenpinscher
1,128,143,174,253,253,253,43,7,12,15,...,50,60,47,48,50,39,38,42,41,affenpinscher
2,25,13,157,73,70,65,60,39,54,38,...,26,84,74,64,65,55,61,51,48,affenpinscher
3,125,125,137,149,142,154,151,146,158,148,...,182,116,149,114,155,133,110,121,78,affenpinscher
4,55,57,63,62,55,80,69,62,152,72,...,98,90,35,66,90,59,72,50,52,affenpinscher
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,42,60,32,30,50,36,61,62,88,44,...,123,112,135,140,130,130,118,105,122,airedale
146,104,102,112,104,105,104,106,97,98,98,...,165,141,140,193,125,155,138,162,126,airedale
147,22,14,11,15,15,15,12,25,39,24,...,115,83,46,47,38,97,123,56,194,airedale
148,110,100,94,87,94,77,78,88,82,100,...,79,47,49,63,79,32,69,60,65,airedale


In [24]:
df.columns = df.columns.astype(str)
df.to_parquet(data_raw_file)

In [36]:
 picture_obj.json()['classification']

'affenpinscher'